# Customer.IO Data Pipelines API - Authentication and Utilities

## Purpose

This notebook demonstrates how to authenticate with Customer.IO's Data Pipelines API and use the utility modules we've created.
It covers API client initialization, testing connections, and demonstrates core functionality with real examples.

## Prerequisites

- Complete setup from `00_setup_and_configuration.ipynb`
- Customer.IO API key configured in Databricks secrets
- Utility modules (`utils/`) available in the Python path

## Key Concepts

- **API Authentication**: Using API keys with Basic Auth
- **Rate Limiting**: Handling 3000 requests per 3 seconds limit
- **Error Handling**: Retry logic and graceful degradation
- **Request Validation**: Using Pydantic models for data validation
- **Data Transformation**: Converting between formats for API consumption

## Setup and Imports

In [ ]:
# Import standard libraries
import sys
import os
from datetime import datetime, timezone
from typing import Dict, List, Optional, Any
import json
import time

# Add utils directory to Python path
sys.path.append('/Workspace/Repos/customer_io_notebooks/utils')

# Import Customer.IO utilities
from utils.api_client import CustomerIOClient
from utils.validators import (
    IdentifyRequest, 
    TrackRequest, 
    GroupRequest,
    validate_request_size,
    create_context
)
from utils.transformers import (
    CustomerTransformer,
    EventTransformer,
    BatchTransformer,
    ContextTransformer
)
from utils.error_handlers import (
    CustomerIOError,
    RateLimitError,
    ValidationError,
    NetworkError,
    retry_on_error,
    ErrorContext
)

# Databricks and Spark imports
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from delta.tables import DeltaTable

# Validation and logging
import structlog
from pydantic import ValidationError as PydanticValidationError

# Initialize logger
logger = structlog.get_logger("customerio_demo")

print("SUCCESS: All imports successful")

## Configuration and Client Initialization

In [ ]:
# Get configuration from widgets (set in previous notebook)
CUSTOMERIO_API_KEY = dbutils.widgets.get("customerio_api_key") or "test_key_demo_12345"
CUSTOMERIO_REGION = dbutils.widgets.get("customerio_region") or "us"
DATABASE_NAME = dbutils.widgets.get("database_name") or "customerio_demo"
CATALOG_NAME = dbutils.widgets.get("catalog_name") or "main"
ENVIRONMENT = dbutils.widgets.get("environment") or "test"

print(f"Configuration:")
print(f"  Region: {CUSTOMERIO_REGION}")
print(f"  Database: {CATALOG_NAME}.{DATABASE_NAME}")
print(f"  Environment: {ENVIRONMENT}")
print(f"  API Key: {'*' * (len(CUSTOMERIO_API_KEY) - 4) + CUSTOMERIO_API_KEY[-4:]}")

# Use current database
spark.sql(f"USE {CATALOG_NAME}.{DATABASE_NAME}")

## Initialize Customer.IO API Client

In [ ]:
# Initialize the Customer.IO client with proper configuration
try:
    client = CustomerIOClient(
        api_key=CUSTOMERIO_API_KEY,
        region=CUSTOMERIO_REGION,
        timeout=30,
        max_retries=3,
        retry_backoff_factor=2.0,
        enable_logging=True,
        spark_session=spark
    )
    
    print("SUCCESS: Customer.IO client initialized successfully")
    print(f"   Base URL: {client.base_url}")
    print(f"   Rate Limit: {client.rate_limit.max_requests} requests per {client.rate_limit.window_seconds} seconds")
    print(f"   Max Retries: {client.max_retries}")
    print(f"   Logging Enabled: {client.enable_logging}")
    
except Exception as e:
    print(f"ERROR: Failed to initialize Customer.IO client: {str(e)}")
    raise

## Test API Connection and Authentication

In [ ]:
# Test API connection with health check
print("Testing API connection...")

try:
    # Test with region endpoint (safest test call)
    if ENVIRONMENT != "test":  # Only make real API calls in non-test environments
        region_info = client.get_region()
        print(f"SUCCESS: API connection successful")
        print(f"   Region response: {region_info}")
        
        # Test health check method
        is_healthy = client.health_check()
        print(f"   Health check: {'SUCCESS: Healthy' if is_healthy else 'ERROR: Unhealthy'}")
    else:
        print("WARNING: Running in test mode - skipping actual API calls")
        print("   Client configured correctly for when real API key is available")
        
except CustomerIOError as e:
    print(f"ERROR: Customer.IO API Error: {str(e)}")
    if hasattr(e, 'status_code'):
        print(f"   Status Code: {e.status_code}")
except NetworkError as e:
    print(f"ERROR: Network Error: {str(e)}")
except Exception as e:
    print(f"ERROR: Unexpected Error: {str(e)}")

## Demonstrate Request Validation with Pydantic Models

In [ ]:
# Example 1: Valid identify request
print("=== Testing Request Validation ===")
print("\n1. Valid Identify Request:")

try:
    valid_identify = IdentifyRequest(
        userId="user_12345",
        traits={
            "email": "user@example.com",
            "first_name": "John",
            "last_name": "Doe",
            "plan": "premium",
            "signup_date": "2024-01-15"
        },
        timestamp=datetime.now(timezone.utc)
    )
    print(f"SUCCESS: Valid request created: {valid_identify.dict()}")
except PydanticValidationError as e:
    print(f"ERROR: Validation error: {e}")

# Example 2: Invalid identify request (missing user identification)
print("\n2. Invalid Identify Request (missing user ID):")

try:
    invalid_identify = IdentifyRequest(
        traits={"email": "user@example.com"}
    )
    print(f"SUCCESS: Request created: {invalid_identify.dict()}")
except PydanticValidationError as e:
    print(f"ERROR: Expected validation error: {e}")

# Example 3: Valid track request
print("\n3. Valid Track Request:")

try:
    valid_track = TrackRequest(
        userId="user_12345",
        event="Product Viewed",
        properties={
            "product_id": "prod_abc123",
            "product_name": "Awesome Widget",
            "category": "electronics",
            "price": 29.99,
            "currency": "USD"
        },
        timestamp=datetime.now(timezone.utc)
    )
    print(f"SUCCESS: Valid track request: {valid_track.dict()}")
except PydanticValidationError as e:
    print(f"ERROR: Validation error: {e}")

# Example 4: Invalid track request (empty event name)
print("\n4. Invalid Track Request (empty event name):")

try:
    invalid_track = TrackRequest(
        userId="user_12345",
        event="",  # Empty event name
        properties={"test": "value"}
    )
    print(f"SUCCESS: Request created: {invalid_track.dict()}")
except PydanticValidationError as e:
    print(f"ERROR: Expected validation error: {e}")

## Data Transformation Examples

In [ ]:
# Load sample customer data from Delta table
print("=== Data Transformation Examples ===")
print("\n1. Loading sample customer data:")

customers_df = spark.table(f"{CATALOG_NAME}.{DATABASE_NAME}.customers").limit(5)
customers_df.show(5, truncate=False)

print(f"Total customers in table: {spark.table(f'{CATALOG_NAME}.{DATABASE_NAME}.customers').count()}")

In [ ]:
# Transform Spark DataFrame to identify requests
print("\n2. Transform to Customer.IO identify requests:")

# Get a small sample for demonstration
sample_customers = spark.table(f"{CATALOG_NAME}.{DATABASE_NAME}.customers").limit(3)

# Transform to identify requests
identify_requests = CustomerTransformer.spark_to_identify_requests(
    df=sample_customers,
    user_id_col="user_id",
    email_col="email",
    traits_cols=["custom_attributes"],  # Use the map column as traits
    timestamp_col="created_at"
)

print(f"Generated {len(identify_requests)} identify requests:")
for i, request in enumerate(identify_requests[:2]):  # Show first 2
    print(f"  Request {i+1}: {json.dumps(request, indent=2, default=str)}")

In [ ]:
# Transform event data to track requests
print("\n3. Transform events to track requests:")

# Get sample event data
sample_events = spark.table(f"{CATALOG_NAME}.{DATABASE_NAME}.events").limit(3)
sample_events.show(3, truncate=False)

# Transform to track requests
track_requests = EventTransformer.spark_to_track_requests(
    df=sample_events,
    user_id_col="user_id",
    event_name_col="event_name",
    properties_cols=["properties"],  # Use the map column as properties
    timestamp_col="timestamp"
)

print(f"\nGenerated {len(track_requests)} track requests:")
for i, request in enumerate(track_requests[:2]):  # Show first 2
    print(f"  Request {i+1}: {json.dumps(request, indent=2, default=str)}")

## Create Specialized Event Types

In [ ]:
# Create ecommerce events using transformers
print("=== Creating Specialized Events ===")
print("\n1. Ecommerce Events:")

# Product viewed event
product_viewed = EventTransformer.create_ecommerce_event(
    event_name="Product Viewed",
    user_id="user_demo_001",
    product_id="prod_widget_123",
    price=29.99,
    quantity=1,
    currency="USD",
    # Additional properties
    product_name="Amazing Widget",
    category="electronics",
    brand="WidgetCorp",
    sku="WDG-123-RED"
)

print("Product Viewed Event:")
print(json.dumps(product_viewed, indent=2))

# Order completed event
order_completed = EventTransformer.create_ecommerce_event(
    event_name="Order Completed",
    user_id="user_demo_001",
    order_id="order_789",
    price=89.97,  # Total order value
    currency="USD",
    # Additional properties
    products=[
        {"product_id": "prod_widget_123", "quantity": 3, "price": 29.99}
    ],
    payment_method="credit_card",
    shipping_method="standard"
)

print("\nOrder Completed Event:")
print(json.dumps(order_completed, indent=2))

In [ ]:
# Create mobile app events
print("\n2. Mobile App Events:")

# Application opened event
app_opened = EventTransformer.create_mobile_app_event(
    event_name="Application Opened",
    user_id="user_mobile_001",
    app_version="2.1.0",
    os_version="iOS 17.2",
    device_model="iPhone 15 Pro",
    # Additional properties
    session_id="session_abc123",
    from_push_notification=False,
    app_build="2100"
)

print("Application Opened Event:")
print(json.dumps(app_opened, indent=2))

# Screen viewed event
screen_viewed = EventTransformer.create_mobile_app_event(
    event_name="Screen Viewed",
    user_id="user_mobile_001",
    app_version="2.1.0",
    # Additional properties
    screen_name="Product Details",
    screen_category="ecommerce",
    product_id="prod_widget_123"
)

print("\nScreen Viewed Event:")
print(json.dumps(screen_viewed, indent=2))

## Context Creation Examples

In [ ]:
# Create context objects for different platforms
print("=== Context Creation Examples ===")
print("\n1. Web Context:")

web_context = ContextTransformer.create_web_context(
    ip="192.168.1.100",
    user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36",
    url="https://example.com/products/widget-123",
    referrer="https://google.com/search",
    locale="en-US",
    timezone="America/New_York"
)

print(json.dumps(web_context, indent=2))

print("\n2. Mobile Context:")

mobile_context = ContextTransformer.create_mobile_context(
    app_name="CustomerIO Demo App",
    app_version="2.1.0",
    os_name="iOS",
    os_version="17.2",
    device_model="iPhone 15 Pro",
    device_id="device_12345",
    locale="en-US",
    timezone="America/Los_Angeles"
)

print(json.dumps(mobile_context, indent=2))

## Batch Operations and Size Optimization

In [ ]:
# Create batch requests and optimize sizes
print("=== Batch Operations ===")
print("\n1. Creating batch requests:")

# Create multiple requests
sample_identify = [
    {
        "userId": f"user_batch_{i}",
        "traits": {
            "email": f"user{i}@example.com",
            "plan": "premium" if i % 2 == 0 else "basic",
            "signup_date": "2024-01-15"
        }
    }
    for i in range(5)
]

sample_track = [
    {
        "userId": f"user_batch_{i}",
        "event": "Page Viewed",
        "properties": {
            "page_name": "Home",
            "url": f"https://example.com/page-{i}"
        }
    }
    for i in range(3)
]

# Create batch request
batch_requests = BatchTransformer.create_batch_request(
    identify_requests=sample_identify,
    track_requests=sample_track,
    max_batch_size=10
)

print(f"Created {len(batch_requests)} batch(es)")
for i, batch in enumerate(batch_requests):
    batch_size = BatchTransformer.estimate_batch_size(batch["batch"])
    print(f"  Batch {i+1}: {len(batch['batch'])} requests, ~{batch_size} bytes")

# Show first batch structure
if batch_requests:
    print(f"\nFirst batch preview (first 2 requests):")
    preview_batch = {"batch": batch_requests[0]["batch"][:2]}
    print(json.dumps(preview_batch, indent=2, default=str))

In [ ]:
# Demonstrate batch size optimization
print("\n2. Batch size optimization:")

# Create requests with varying sizes
large_requests = []
for i in range(20):
    # Create request with large properties to test size limits
    large_request = {
        "userId": f"user_large_{i}",
        "event": "Data Import",
        "properties": {
            "large_data": "x" * 1000,  # 1KB of data per request
            "import_id": f"import_{i}",
            "batch_number": i // 5
        }
    }
    large_requests.append(large_request)

# Optimize batch sizes to stay within limits
optimized_batches = BatchTransformer.optimize_batch_sizes(
    requests=large_requests,
    max_size_bytes=10 * 1024  # 10KB limit for demo (normally 500KB)
)

print(f"Optimized into {len(optimized_batches)} batches:")
for i, batch in enumerate(optimized_batches):
    batch_size = BatchTransformer.estimate_batch_size(batch)
    print(f"  Optimized Batch {i+1}: {len(batch)} requests, {batch_size} bytes")

## Error Handling Demonstrations

In [ ]:
# Demonstrate error handling patterns
print("=== Error Handling Demonstrations ===")
print("\n1. Error Context Manager:")

# Example of graceful error handling
with ErrorContext(
    operation_name="demo_api_call",
    logger=logger,
    raise_on_error=False,
    default_return={"status": "failed", "message": "Operation failed gracefully"}
) as error_ctx:
    # Simulate an operation that might fail
    if ENVIRONMENT == "test":
        # Simulate a controlled error for demonstration
        raise CustomerIOError("Simulated API error for demonstration")
    else:
        print("Real API call would happen here")

# Check if error occurred and get default result
if error_ctx.error:
    result = error_ctx.get_result()
    print(f"Error handled gracefully: {result}")
else:
    print("Operation completed successfully")

In [ ]:
# Demonstrate retry decorator
print("\n2. Retry Decorator Example:")

@retry_on_error(max_retries=2, backoff_factor=1.5)
def demo_api_call_with_retry(should_fail: bool = False):
    """Demo function that can be configured to fail for testing."""
    print(f"  Attempting API call (fail={should_fail})...")
    
    if should_fail:
        # Simulate different types of failures
        import random
        error_type = random.choice(["network", "server", "rate_limit"])
        
        if error_type == "network":
            raise NetworkError("Simulated network error")
        elif error_type == "server":
            raise CustomerIOError("Simulated server error", status_code=500)
        else:
            raise RateLimitError("Simulated rate limit error", retry_after=1)
    
    return {"status": "success", "message": "API call completed"}

# Test successful call
try:
    result = demo_api_call_with_retry(should_fail=False)
    print(f"SUCCESS: Success: {result}")
except Exception as e:
    print(f"ERROR: Failed: {str(e)}")

# Test failing call (will retry)
try:
    result = demo_api_call_with_retry(should_fail=True)
    print(f"SUCCESS: Success after retries: {result}")
except Exception as e:
    print(f"ERROR: Failed after retries: {str(e)}")

## Rate Limiting Demonstration

In [ ]:
# Demonstrate rate limiting behavior
print("=== Rate Limiting Demonstration ===")

# Check current rate limit status
print(f"Rate limit configuration:")
print(f"  Max requests: {client.rate_limit.max_requests}")
print(f"  Window: {client.rate_limit.window_seconds} seconds")
print(f"  Current requests: {client.rate_limit.current_requests}")
print(f"  Window start: {client.rate_limit.window_start}")

# Simulate rate limit checking
print(f"\nRate limit status:")
print(f"  Can make request: {client.rate_limit.can_make_request()}")
print(f"  Time until reset: {client.rate_limit.time_until_reset():.2f} seconds")

# Simulate some requests to show rate limiting in action
print(f"\nSimulating request tracking:")
for i in range(5):
    if client.rate_limit.can_make_request():
        client.rate_limit.record_request()
        print(f"  Request {i+1}: SUCCESS: Allowed (total: {client.rate_limit.current_requests})")
    else:
        print(f"  Request {i+1}: ERROR: Rate limited")
        break

## Request Size Validation

In [ ]:
# Demonstrate request size validation
print("=== Request Size Validation ===")

# Test normal-sized request
normal_request = {
    "userId": "user_123",
    "event": "Page Viewed",
    "properties": {
        "page_name": "Home",
        "url": "https://example.com"
    }
}

is_valid_size = validate_request_size(normal_request)
print(f"Normal request size valid: {is_valid_size}")
print(f"  Request size: ~{len(json.dumps(normal_request).encode())} bytes")

# Test oversized request
oversized_request = {
    "userId": "user_123",
    "event": "Large Data Import",
    "properties": {
        "large_payload": "x" * (33 * 1024),  # 33KB - exceeds 32KB limit
        "import_id": "large_import_001"
    }
}

is_oversized_valid = validate_request_size(oversized_request)
print(f"\nOversized request size valid: {is_oversized_valid}")
print(f"  Request size: ~{len(json.dumps(oversized_request).encode())} bytes")
print(f"  Limit: {32 * 1024} bytes (32KB)")

## Clean Up and Summary

In [ ]:
# Clean up resources
print("=== Clean Up ===")

# Close the API client connection
client.close()
print("SUCCESS: API client connection closed")

# Summary of what we accomplished
print("\n=== Summary ===")
print("This notebook demonstrated:")
print("SUCCESS: Customer.IO API client initialization and configuration")
print("SUCCESS: API authentication and connection testing")
print("SUCCESS: Request validation using Pydantic models")
print("SUCCESS: Data transformation from Spark DataFrames to API requests")
print("SUCCESS: Creating specialized ecommerce and mobile events")
print("SUCCESS: Context object creation for different platforms")
print("SUCCESS: Batch operations and size optimization")
print("SUCCESS: Error handling patterns and retry logic")
print("SUCCESS: Rate limiting demonstration")
print("SUCCESS: Request size validation")

print("\nCOMPLETED: Ready to proceed to people management operations!")

## Next Steps

This notebook has successfully demonstrated the Customer.IO API client and utility modules:

### Key Accomplishments:

SUCCESS: **API Client Initialized** - Production-ready client with rate limiting and error handling

SUCCESS: **Request Validation** - Pydantic models ensure data integrity before API calls

SUCCESS: **Data Transformation** - Seamless conversion from Spark DataFrames to API formats

SUCCESS: **Specialized Events** - Easy creation of ecommerce and mobile app events

SUCCESS: **Context Management** - Platform-specific context objects for rich event data

SUCCESS: **Batch Operations** - Optimized batching with size limits and intelligent splitting

SUCCESS: **Error Handling** - Comprehensive error handling with retry logic and graceful degradation

SUCCESS: **Rate Limiting** - Built-in protection against API rate limits

### Ready for Next Notebooks:

1. **02_people_management.ipynb** - User identification, deletion, and lifecycle management
2. **03_events_and_tracking.ipynb** - Event tracking and custom event implementation  
3. **04_objects_and_relationships.ipynb** - Group/company management and relationships

### Key Utilities Available:

- **CustomerIOClient**: Production-ready API client with all features
- **Validation Models**: Pydantic models for all API request types
- **Transformers**: Data conversion utilities for Spark/Pandas to API formats
- **Error Handlers**: Comprehensive error handling and retry mechanisms

The foundation is now solid for building comprehensive Customer.IO integrations in Databricks!